# Обучение с подкреплением

https://qudata.com/ml/ru/RL_Reinforcement_learning.html

Обучение с подкреплением (Reinforcement Learning, RL) - набор методов, позволяющих агенту (интеллектуальной системе) вырабатывать оптимальную стратегию при его взаимодействии со средой (внешним миром). 

# Траектория:
$$\tau = (s_0,a_0,r_0),\dots,(s_T,a_T,r_T) ,$$
где $s_i$ состояние среды на шаге $i$, $a_i$ воздействие агента на среду на шаге $i$,
$r_i$ вознаграждение на шаге $i$.

Вознаграждение начиная с шага $t$:
$$R_t(\tau)=\sum\limits_{j=t}^{T}\gamma^{j-t}r_{j}, $$
$$R(\tau) = R_0(\tau) $$
где $\gamma$ дисконтный множитель (коэффициент значимости вознаграждения при переходе на следущий шаг).

Обучение с подкреплением - набор методов, позволяющих агенту (интеллектуальной системе) вырабатывать оптимальную стратегию при его взаимодействии со средой (внешним миром). 

Среда предоставляет информацию, описывающую состояние системы. Агент взаимодействует со средой, наблюдая состояние и используя данную информацию при выборе действия. Среда принимает действие и переходит в следующее состояние, а затем возвращает агенту следующее состояние и вознаграждение. Когда цикл «состояние → действие → вознаграждение» завершен, предполагается, что сделан один шаг. Цикл повторяется, пока среда не завершится, например, когда задача решена.

Стратегия $\pi$ — это функция, отображающая состояния вероятности действий, которые используются для выбора действия $a \sim \pi(s) $. 

Стратегия $\pi$ содержит настраеваемые параметры $\omega$, чтобы это подчеркнуть будем писать $\pi_{\omega}(s)$.

Целевая функция — это ожидаемая отдача по всем полным траекториям, порожденным агентом.
$$J(\omega)=J(\pi_{\omega})=M_{\tau\sim \omega}R(\tau),$$
где $M_{\tau\sim \omega}$ математическое ожидание по всех траекториям $\tau$ соответствующим значениям параметров $\omega$.

Задача:
$$J(\omega)\to \max $$

$$\nabla_{\omega}J=\nabla_{\omega} M_{\tau\sim \omega}R(\tau)$$

$$\nabla_{\omega}M_{\tau\sim \omega}R(\tau)=\nabla_{\omega}\int R(\tau)p(\tau\mid\omega)d\tau= $$

$$=\int \nabla_{\omega}(R(\tau)p(\tau\mid\omega))d\tau =\int (\nabla_{\omega}(R(\tau))p(\tau\mid\omega)+R(\tau)\nabla_{\omega}(p(\tau\mid\omega)))d\tau =$$

$$ =\int R(\tau)\nabla_{\omega}(p(\tau\mid\omega))d\tau =\int R(\tau)\nabla_{\omega}(p(\tau\mid\omega)) \dfrac{p(\tau\mid\omega)}{p(\tau\mid\omega)}d\tau =$$

$$ =\int R(\tau)\dfrac{\nabla_{\omega}(p(\tau\mid\omega)) }{p(\tau\mid\omega)}p(\tau\mid\omega)d\tau =\int R(\tau)\nabla_{\omega}(\ln p(\tau\mid\omega))p(\tau\mid\omega)d\tau =$$

$$ = M_{\tau\sim \omega}(R(\tau)\nabla_{\omega}(\ln p(\tau\mid\omega)))$$

$$p(\tau\mid\omega)=\prod\limits_t p(s_{s+1}\mid s_t,a_t)\pi_{\omega}(a_t\mid s_t) $$

$$\ln p(\tau\mid\omega)=\ln\prod\limits_t p(s_{s+1}\mid s_t,a_t)\pi_{\omega}(a_t\mid s_t) = \sum\limits_t (\ln p(s_{s+1}\mid s_t,a_t)+\ln\pi_{\omega}(a_t\mid s_t))$$

$$\nabla_{\omega}\ln p(\tau\mid\omega)=\nabla_{\omega}\sum\limits_t (\ln p(s_{s+1}\mid s_t,a_t)+\ln\pi_{\omega}(a_t\mid s_t))=\nabla_{\omega}\sum\limits_t \ln\pi_{\omega}(a_t\mid s_t)$$



$$Loss(\omega) = - R(\tau)\sum\limits_t \ln\pi_{\omega}(a_t\mid s_t) $$

In [44]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np
import pandas as pd
import time
import random

In [45]:
DL = 9

In [46]:
def space():
    return torch.tensor(np.random.permutation(DL)) + 1.

In [47]:
space()

tensor([6., 1., 7., 5., 2., 4., 9., 8., 3.])

In [48]:
Prs = 10
model = torch.nn.Sequential(
        torch.nn.Linear(DL,Prs),
        torch.nn.Sigmoid(),
        torch.nn.Linear(Prs, DL),
        torch.nn.Softmax()
        )

In [49]:
x = space()

model(x)

tensor([0.0719, 0.1002, 0.0852, 0.1098, 0.0775, 0.1424, 0.1476, 0.1495, 0.1159],
       grad_fn=<SoftmaxBackward0>)

In [50]:
x

tensor([5., 3., 7., 4., 8., 6., 2., 9., 1.])

In [51]:
optimizer = torch.optim.Adam(model.parameters())

In [52]:
def bolvan(x):
    x[np.random.randint(DL)] = -1.
    return x

In [61]:
time_1 = time.time()

gamma = 1
alpha = 0.9
max_ep = 1000

for episode in range(max_ep):
    SPACE = space()
    tr = []
    if episode%2:
        SPACE = bolvan(SPACE)

    while SPACE.sum() + DL:

        probs = model(SPACE)
        m = Categorical(probs)
        action = m.sample()
        reward = SPACE[action]
        tr.append((SPACE.clone(), action, reward))
        SPACE[action] = -1.

        SPACE = bolvan(SPACE)

    loss = 0.
    T = len(tr)
    for t in range(T):
        R = 0.
        for i in range(t,T):
            R += (gamma**(i - t))*tr[i][2]

        loss += -alpha*R*Categorical(model(tr[t][0])).log_prob(tr[t][1])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


print(time.time() - time_1)

C:\Users\kuzin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


8
8
20
8
7
8
10
8
9
6
6
9
10
7
9
7
7
10
8
6
11
6
8
8
10
8
7
9
6
8
9
14
10
5
6
7
10
10
6
9
8
5
14
13
6
7
7
5
7
8
6
6
9
7
7
7
6
14
7
16
7
7
7
8
8
6
10
6
7
12
10
8
8
10
8
13
7
5
7
7
12
10
10
7
8
6
8
6
8
10
6
7
9
7
7
8
7
5
6
6
7
10
7
7
8
9
7
9
7
5
7
11
5
5
13
7
8
10
9
6
9
6
10
7
14
5
9
6
8
5
7
10
9
6
7
6
8
7
6
10
6
5
6
7
8
6
9
5
9
10
9
5
9
8
7
6
6
7
8
6
6
7
13
16
7
8
8
5
10
8
6
6
8
12
7
7
9
11
6
7
9
9
9
8
6
9
8
11
7
7
8
5
10
5
8
9
10
6
11
8
8
6
6
8
13
6
9
7
7
9
6
8
9
6
8
11
8
8
8
5
8
8
9
7
10
12
9
7
8
7
7
6
7
12
9
8
14
8
10
11
9
6
11
8
7
7
11
8
9
10
10
7
6
6
8
10
6
8
8
8
12
5
9
7
8
7
6
8
5
7
13
7
10
9
8
11
6
6
5
6
6
5
8
6
12
8
11
6
7
8
9
9
8
5
9
9
8
9
11
10
13
6
16
6
7
9
13
6
7
9
9
6
7
7
11
6
5
10
9
7
13
10
8
6
9
6
7
8
7
8
10
8
6
8
6
9
10
7
9
11
9
9
8
12
11
6
10
5
9
7
7
6
6
6
6
7
8
5
12
7
6
7
10
8
7
9
6
7
10
5
9
7
9
8
6
11
9
7
13
8
9
6
9
12
13
14
8
7
12
11
7
5
5
5
5
10
11
4
8
5
14
6
9
11
6
10
12
8
6
6
8
5
7
6
9
6
8
7
6
6
7
7
6
7
6
12
6
6
7
11
7
5
6
8
6
6
7
7
8
11
9
6
6
10
9
7
11
10
8
13
8


KeyboardInterrupt: 

In [54]:
S = space()
print(S)
model(S)

tensor([7., 5., 1., 3., 2., 9., 8., 6., 4.])


tensor([0.0585, 0.0893, 0.0693, 0.1153, 0.1045, 0.1077, 0.1524, 0.1722, 0.1308],
       grad_fn=<SoftmaxBackward0>)

In [55]:
time_1 = time.time()


alpha = 0.9
max_ep = 1000

for episode in range(max_ep):
    SPACE = space()
    if episode%2:
        SPACE = bolvan(SPACE)

    while SPACE.sum() + DL:


        probs = model(SPACE)
        m = Categorical(probs)
        action = m.sample()
        reward = SPACE[action]
        loss = -alpha*reward*m.log_prob(action)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        SPACE[action] = -1.

        SPACE = bolvan(SPACE)


print(time.time() - time_1)

15.620139122009277


In [56]:
time_1 = time.time()

max_ep = 100
R = 0
for episode in range(max_ep):
    SPACE = space()
    if episode%2:
        SPACE = bolvan(SPACE)

    while SPACE.sum() + DL:


        probs = model(SPACE)
        action = probs.argmax()
        reward = SPACE[action]
        R += reward


        SPACE[action] = -1.

        SPACE = bolvan(SPACE)


print(time.time() - time_1)
print(R/max_ep)

0.17707204818725586
tensor(33.2100)


$$SoftMax(x_1,x_2,\dots,x_n)=\left( \frac{e^{x_1}}{\sum\limits_{i=1}^{n}e^{x_i}}, \frac{e^{x_2}}{\sum\limits_{i=1}^{n}e^{x_i}},\dots, \frac{e^{x_n}}{\sum\limits_{i=1}^{n}e^{x_i}}\right) $$

In [57]:
def softmax(x):
    return np.exp(x)/(np.exp(x).sum())

In [58]:
softmax([-1,2,3])

array([0.01321289, 0.26538793, 0.72139918])

In [59]:
torch.nn.functional.softmax(torch.tensor([-1.,2.,3.]))

C:\Users\kuzin\AppData\Local\Temp\ipykernel_10284\1649720306.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.nn.functional.softmax(torch.tensor([-1.,2.,3.]))


tensor([0.0132, 0.2654, 0.7214])